In [26]:
from urllib import request
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

def scrape_items(main_url,max_page):
    links = []
    req = requests.get(main_url,headers=headers)
    page = bs4.BeautifulSoup(req.content, "lxml")

    items = page.find_all('li')
    for item in items:
        link = item.find('a', class_='db7c79')
        if link:
            href = link.get('href')
            links.append(href)

    for nb in range(2,max_page+1):
        print(nb)
        url = main_url+'?page='+str(nb)
        req = requests.get(url,headers=headers)
        page = bs4.BeautifulSoup(req.content, "lxml")

        items = page.find_all('li') #,class_='product-item'
        for item in items:
            link = item.find('a', class_='db7c79')
            if link:
                href = link.get('href')
                links.append(href)
    return links

In [48]:
from bs4 import BeautifulSoup
def scrape_details(links):
    material_lists = []
    names = []
    prices = []
    counts = []
    i = 0
    for link in links:
        if i%100 == 0:
            print(i)
        i+=1
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.content, 'lxml')
        if not soup:
            names.append('NA')
            prices.append('NA')
            material_lists.append('NA')
            counts.append('NA')
            continue
        #name
        name = soup.find('h1')
        if not name:
            names.append('NA')
            prices.append('NA')
            material_lists.append('NA')
            counts.append('NA')
            continue
        nameTEXT = name.text

        #counting nb of variations
        different = soup.find_all('li', class_='list-item')
        count = len(different)
        
        
        #price
        price_div = soup.find('div', class_='price parbase')
        if not price_div:
            names.append('NA')
            prices.append('NA')
            material_lists.append('NA')
            counts.append('NA')
            continue
        price_span = price_div.find('span')
        price_value = price_span.text.strip()
        
        # materials
        h3 = soup.find(lambda tag: tag.name == 'h3' and "Explications relatives " in tag.text) #  
        parent_tag = h3.find_next_sibling()

        # its under dt 
        dt_tags = parent_tag.find_all('dt')
        materials = []

        
        # get the text from the dt tags
        for dt in dt_tags:
            materials.append(dt.text)

        names.append(nameTEXT)
        prices.append(price_value)
        material_lists.append(materials)
        counts.append(count)
    return names, prices, material_lists,counts

In [ ]:
def run_selenium(unique_links):
    driver = webdriver.Chrome()
    informations = []
    for url in unique_links:
        driver.get(url)
        try:
            materials_button =  WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.ID,"toggle-materialsAndSuppliersAccordion")))
            materials_button.click()
            
            popup_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.CLASS_NAME, "bbcd86")))
            popup_button.click()

            popup_elements = WebDriverWait(driver, 4).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "dd5fe7")))
            popup_contents = [element.text.strip() for element in popup_elements]
        except:
            popup_contents = []
            informations.append(popup_contents)
            continue
        informations.append(popup_contents)

    driver.quit()
    return informations

In [41]:
main_url = 'https://www2.hm.com/fr_fr/femme/catalogue-par-produit/pantalons.html'
max_page = 18
links = scrape_items(main_url,max_page)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [42]:
unique_links = list(set(links))
(len(unique_links),len(links))

(643, 4185)

In [49]:
names,prices,material_lists,counts = scrape_details(unique_links)

0
100
200
300
400
500
600


In [44]:
informations = run_selenium(unique_links)

In [51]:
fixed_material_lists = [[s.encode('utf-8').decode('unicode_escape') for s in sublist] for sublist in material_lists]

print(fixed_material_lists)

print(len(names), len(prices), len(fixed_material_lists), len(unique_links))
data = {
    'Product Type': ['Pantalons']*len(names),
    'Product Name': names,
    'Price': prices,
    'Material': fixed_material_lists,
    'Link': unique_links[:len(names)],
    'Informations': informations[:len(names)]
}


df = pd.DataFrame(data)


df.to_csv('hmPANTALONS.csv', index=False)

[[' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Polyester'], [' Polyester', ' Élasthanne'], [' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Polyester'], [' Coton', ' Élasthanne'], [' Coton'], [' Viscose', ' Viscose LivaEco™', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester'], [' Coton', ' Coton recyclé'], [' Polyester', ' Polyester recyclé', ' Viscose', ' Viscose LivaEco™', ' Élasthanne'], [' Lin', " Polyester recyclé d'origine textile", ' Viscose', ' Viscose LivaEco™'], [' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Polyamide', ' Polyamide recyclé', ' Élasthanne'], [' Polyamide', ' Polyamide recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Polyamide', ' Polyester', ' Polyester recyclé'], [' Polyamide', ' Polyamide recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester'], [' Coton', ' Coton biologique', ' Viscose', ' Viscose LivaEco™', ' Élasthanne'], [' Laine', ' Lyoc

In [52]:
main_url = 'https://www2.hm.com/fr_fr/femme/catalogue-par-produit/sweatshirts.html'
max_page = 5
links = scrape_items(main_url,max_page)
unique_links = list(set(links))
names,prices,material_lists,counts = scrape_details(unique_links)
informations = run_selenium(unique_links)
fixed_material_lists = [[s.encode('utf-8').decode('unicode_escape') for s in sublist] for sublist in material_lists]

print(fixed_material_lists)

print(len(names), len(prices), len(fixed_material_lists), len(unique_links))
data = {
    'Product Type': ['Sweatshirt']*len(names),
    'Product Name': names,
    'Price': prices,
    'Material': fixed_material_lists,
    'Link': unique_links[:len(names)],
    'Informations': informations[:len(names)]
}


df = pd.DataFrame(data)


df.to_csv('hmSWEATSHIRTS.csv', index=False)

2
3
4
5
0
100
[[' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Polyester', ' Polyester recyclé'], [' Coton', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Polyester'], [' Coton', ' Polyester'], [' Coton', ' Polyester', ' Viscose'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Viscose'], [' Coton', ' Coton recyclé', ' Polyester'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton biologique'], [' Coto

In [53]:
main_url = 'https://www2.hm.com/fr_fr/femme/catalogue-par-produit/jeans.html'
max_page = 8
links = scrape_items(main_url,max_page)
unique_links = list(set(links))
names,prices,material_lists,counts = scrape_details(unique_links)
informations = run_selenium(unique_links)
fixed_material_lists = [[s.encode('utf-8').decode('unicode_escape') for s in sublist] for sublist in material_lists]

print(fixed_material_lists)

print(len(names), len(prices), len(fixed_material_lists), len(unique_links))
data = {
    'Product Type': ['Jeans']*len(names),
    'Product Name': names,
    'Price': prices,
    'Material': fixed_material_lists,
    'Link': unique_links[:len(names)],
    'Informations': informations[:len(names)]
}

df = pd.DataFrame(data)


df.to_csv('hmJEANS.csv', index=False)

2
3
4
5
6
7
8
0
100
200
[[' Coton', ' Coton recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Viscose', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton biologique', ' Coton recyclé'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé', ' Élasthanne'], [' Coton', ' Coton recyclé', ' Polyester', ' Polyester recyclé'], [' Coton', ' Coton recyclé', ' Pol